<a href="https://colab.research.google.com/github/alfa7g7/icesi-nlp-20261-curso/blob/main/Sesion1/FabianSF/7_sentiment_analysis_Foxtrot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---

# **Mini-Proyecto NLP: Análisis de Sentimientos Corporativo**
**Maestría en IA Aplicada | Procesamiento de lenguaje natural**

**Grupo: Foxtroot** |
**Raul Echeverry - Esteban Ordoñez - Fabian Salazar Figueroa**

---

### **Introducción y Contexto del Proyecto**

Este notebook constituye la segunda fase del mini-proyecto de procesamiento de lenguaje natural aplicado al ecosistema de contratación pública colombiana (**SECOP II**). Mientras que la fase anterior se centró en la extracción estructural de requisitos, el presente estudio aborda el **Análisis de Sentimientos y Calificación de Reputación Corporativa**.

**Objetivo General:**
Evaluar y comparar tres metodologías de clasificación de sentimientos (Estadística, Semántica y Léxica) sobre un corpus de retroalimentación de contratistas del Estado. El fin último es integrar este pipeline en un **Agente de IA** que permita filtrar propuestas no solo por cumplimiento técnico, sino por antecedentes de desempeño y reputación histórica.

**Metodología y Alcance:**
1.  **Dataset SECOP-Sentiment:** Debido a la naturaleza sensible de los datos reales, se ha construido un *Gold Standard* de **2,000 registros balanceados** mediante técnicas de generación programática, cubriendo 26 patrones semánticos de éxito y fricción contractual.
2.  **Enfoque Multitécnica:** Se implementan y contrastan tres niveles de IA:
    *   **ML Clásico:** Vectorización TF-IDF con Regresión Logística.
    *   **Innovación Semántica:** Uso de *Word Embeddings* de alta dimensionalidad mediante el modelo **`es_core_news_lg`** de spaCy.
    *   **Análisis Léxico:** Evaluación de diccionarios predefinidos (**TextBlob** para español y **VADER** para la replicación en inglés).
3.  **Transferencia de Conocimiento:** Se incluye una sección de replicación del ejercicio académico original (Movie Reviews) para validar la consistencia del pipeline frente a datasets estándar de la industria.


### Configuración del Entorno y Gestión de Dependencias
Implementamos una rutina de **inicialización robusta** para detectar el entorno de ejecución (Colab vs Local).
El objetivo es garantizar que el modelo de lenguaje `es_core_news_lg` (necesario para la vectorización semántica) y las librerías de análisis estén correctamente instaladas antes de iniciar el pipeline.

**Nota sobre el flujo:** La siguiente celda descarga el archivo `requirements.txt` y fuerza un reinicio del kernel de Python (`os.kill`) para limpiar la memoria. Esto permite que las nuevas versiones de las librerías se carguen correctamente desde el inicio.

**Gestión de salida:** Se ha configurado la instalación para omitir advertencias de dependencias no críticas (stderr), garantizando un informe técnico limpio y enfocado exclusivamente en los hallazgos del análisis.

---


## 1. Configuración del Entorno
Garantizamos la instalación silenciosa de `spaCy LG`, `VADER` y `TextBlob`.

In [ ]:
# Configuración de Entorno
import warnings
import os

# Intentamos importar pkg_resources, si falla usamos importlib (más moderno)
try:
    import pkg_resources
except ImportError:
    import importlib.metadata as pkg_resources

warnings.filterwarnings('ignore')

# Detectamos si estamos en Colab de forma más sencilla
IN_COLAB = 'google.colab' in str(get_ipython())

if IN_COLAB:
    print("Entorno Colab detectado. Limpiando archivos previos, instalando y configurando dependencias...")

    # Eliminamos el archivo si existía
    if os.path.exists('requirements.txt'):
        os.remove('requirements.txt')

    # URL de tu repositorio
    RAW_URL = "https://raw.githubusercontent.com/alfa7g7/icesi-nlp-20261-curso/refs/heads/main/requirements.txt"

    # --- INSTALACIÓN ---
    !wget -q {RAW_URL} -O requirements.txt
    !pip install -q -r requirements.txt 2>/dev/null

    # NUEVO: Instalamos librerías específicas para análisis de sentimientos
    !pip install -q textblob datasets 2>/dev/null

    # Descarga del modelo LARGE de spaCy
    !python -m spacy download es_core_news_lg -q 2>/dev/null

    # NUEVO: Descarga de datos necesarios para TextBlob
    !python -m textblob.download_corpora -q > /dev/null 2>&1

    print("\n✅ Instalación de spaCy 'es_core_news_lg', TextBlob y dependencias completada.")
    print("🔄 Reiniciando el entorno para cargar librerías y modelos...")

    # Forzamos el reinicio
    import os
    os.kill(os.getpid(), 9)
else:
    print("Entorno local detectado. No es necesario reinstalar.")

/tmp/ipython-input-25918917.py:7: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources


Entorno Colab detectado. Limpiando archivos previos, instalando y configurando dependencias...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 82.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 541.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 86.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 92.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.3/293.3 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 101.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3

In [25]:
# ---------------------------------------------------------
# LIBRERÍAS DEL PROYECTO (Standard & Third Party)
# ---------------------------------------------------------
import pandas as pd
import numpy as np
import spacy
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from textblob import TextBlob
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# Cargamos el modelo una sola vez para todo el notebook
nlp = spacy.load('es_core_news_lg')

## 2. Contexto de Negocio: Análisis de Reputación de Contratistas
Para dar continuidad al proyecto de SECOP II, simularemos un análisis de la **reputación de empresas contratistas** basado en reseñas de servicios. Esto permite a nuestra IA de licitaciones calificar con qué empresas es mejor asociarse (consorcios).

En esta sección generamos 2,000 registros para igualar el volumen del ejercicio de las películas.

In [1]:
# Plantillas de comentarios positivos (12+1 variantes)
pos_templates = [
    "Excelente cumplimiento en los plazos de entrega.", "Equipo altamente profesional y capacitado.",
    "La implementación fue un éxito total.", "Superaron las expectativas del pliego técnico.",
    "Gran manejo del presupuesto y transparencia.", "Aliado estratégico con mucha experiencia.",
    "Soporte técnico rápido y eficiente.", "Entregables de alta calidad y bien documentados.",
    "Recomendado para contratos de gran escala.", "Innovación constante en sus soluciones.",
    "Cumplieron con todos los niveles de servicio SLA.", "Gestión administrativa impecable.",
    "Comunicación fluida y proactiva.", "Expertos en normatividad de contratación pública.",
    "Entrega anticipada de los hitos del proyecto.", "Calidad humana y técnica excepcional.",
    "Manejo impecable de riesgos contractuales.", "El mejor proveedor en el sector de tecnología.",
    "Resultados sólidos y consistentes.", "Documentación clara y detallada.",
    "Capacidad de respuesta inmediata ante cambios.", "Altos estándares de ciberseguridad.",
    "Eficiencia operativa demostrada.", "Ética profesional en cada fase.",
    "Cumplimiento del 100% de los requisitos."
]

# Plantillas de comentarios negativos (12+1 variantes)
neg_templates = [
    "Retrasos constantes en la entrega de hitos.", "Personal poco capacitado y con errores técnicos.",
    "La implementación fue un fracaso rotundo.", "No cumplieron con los requisitos del pliego.",
    "Falta de transparencia en el manejo de fondos.", "Proveedor poco confiable y sin experiencia.",
    "Soporte técnico lento o inexistente.", "Entregables mediocres y con falta de rigor.",
    "No recomendado para licitaciones complejas.", "Soluciones obsoletas y sin innovación.",
    "Incumplimiento grave de los niveles de servicio.", "Pésima gestión administrativa.",
    "Mala comunicación y falta de respuesta.", "Desconocimiento de la ley de contratación.",
    "Retrasos injustificados que afectaron el cronograma.", "Trato poco profesional y prepotente.",
    "Nula gestión de riesgos en el contrato.", "El peor servicio que hemos contratado.",
    "Resultados inconsistentes y con errores.", "Faltas graves en la documentación técnica.",
    "Incapacidad de resolver problemas críticos.", "Vulnerabilidades de seguridad detectadas.",
    "Ineficiencia operativa y desperdicio de recursos.", "Faltas a la ética en el proceso.",
    "No cumplieron ni el 50% de lo pactado."
]

# Generamos 1000 de cada uno para tener 2000 registros balanceados
df_pos = pd.DataFrame({'review': np.random.choice(pos_templates, 1000), 'label': 'pos'})
df_neg = pd.DataFrame({'review': np.random.choice(neg_templates, 1000), 'label': 'neg'})

df = pd.concat([df_pos, df_neg])
df = shuffle(df).reset_index(drop=True)

print(f"✅ Dataset corporativo escalado: {len(df)} registros balanceados.")
print(df['label'].value_counts())

✅ Dataset corporativo escalado: 2000 registros balanceados.
label
neg    1000
pos    1000
Name: count, dtype: int64


## 3. Preprocesamiento Avanzado con spaCy LG
A diferencia del notebook base, usaremos **Lematización** para normalizar las críticas antes de analizarlas.

In [8]:
# Preprocesamiento (Lematización)
def clean_text(text):
    doc = nlp(text.lower())
    return " ".join([t.lemma_ for t in doc if not t.is_stop and not t.is_punct])

print("Limpiando y lematizando 2,000 registros...")
df['clean_review'] = df['review'].apply(clean_text)

Limpiando y lematizando 2,000 registros...


In [9]:
df[['review', 'clean_review']].head()

,review,clean_review
0,Nula gestión de riesgos en el contrato.,nulo gestión riesgo contrato
1,Excelente cumplimiento en los plazos de entrega.,excelente cumplimiento plazo entrega
2,El peor servicio que hemos contratado.,servicio contratar
3,Entregables de alta calidad y bien documentados.,entregabl alto calidad documentado
4,Superaron las expectativas del pliego técnico.,superar expectativa pliego técnico


## 4. Implementación de la Técnica 1: Modelo Base: Machine Learning Clásico (TF-IDF)
**Técnica:** Bolsa de palabras con pesado de frecuencia inversa de documento.
**¿Por qué usamos `clean_review`?** Porque TF-IDF funciona mejor cuando palabras con la misma raíz están normalizadas (lemas), reduciendo la dispersión de la matriz.

In [18]:
# Dividimos los datos
X_train, X_test, y_train, y_test = train_test_split(df['clean_review'], df['label'], test_size=0.25, random_state=42)

# Vectorización TF-IDF
tfidf = TfidfVectorizer(max_features=2000)
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

# Clasificador
clf_tfidf = LogisticRegression()
clf_tfidf.fit(X_train_tfidf, y_train)

y_pred_tfidf = clf_tfidf.predict(X_test_tfidf)

print("--- REPORTE MODELO 1: TF-IDF (ML Clásico) ---")
print(classification_report(y_test, y_pred_tfidf))

--- REPORTE MODELO 1: TF-IDF (ML Clásico) ---
              precision    recall  f1-score   support

         neg       1.00      1.00      1.00       250
         pos       1.00      1.00      1.00       250

    accuracy                           1.00       500
   macro avg       1.00      1.00      1.00       500
weighted avg       1.00      1.00      1.00       500



## 5. Implementación de la Técnica 2: INNOVACIÓN: Análisis Semántico con Vectores (spaCy LG)
**Técnica:** Representación densa mediante Word Embeddings de 300 dimensiones.
**¿Por qué es innovador?** A diferencia de TF-IDF, que solo cuenta palabras, los vectores capturan la **proximidad semántica**. Aquí usamos el vector promedio de todo el documento generado por el modelo `lg`.

In [19]:
print("Generando vectores semánticos...")
# Transformamos cada reseña en un vector denso de 300 dimensiones
X_vec = np.array([nlp(text).vector for text in df['clean_review']])
y_vec = df['label']

X_train_v, X_test_v, y_train_v, y_test_v = train_test_split(X_vec, y_vec, test_size=0.25, random_state=42)

# Clasificador sobre vectores
clf_vec = LogisticRegression(max_iter=1000)
clf_vec.fit(X_train_v, y_train_v)

y_pred_vec = clf_vec.predict(X_test_v)

print("--- REPORTE MODELO 2: Word Embeddings (Innovación) ---")
print(classification_report(y_test_v, y_pred_vec))

Generando vectores semánticos...
--- REPORTE MODELO 2: Word Embeddings (Innovación) ---
              precision    recall  f1-score   support

         neg       1.00      1.00      1.00       250
         pos       1.00      1.00      1.00       250

    accuracy                           1.00       500
   macro avg       1.00      1.00      1.00       500
weighted avg       1.00      1.00      1.00       500



## 6. Implementación de la Técnica 3: Análisis de Léxico (TextBlob).
En lugar de VADER (que fallará), usaremos **TextBlob** con su analizador de sentimientos. Esta técnica busca palabras en un diccionario y les asigna un valor.

In [22]:
def get_textblob_sentiment(text):
    # TextBlob analiza la polaridad de -1 (muy negativo) a 1 (muy positivo)
    analysis = TextBlob(text)
    # Si usamos clean_review, TextBlob funciona mejor con palabras raíz
    return 'pos' if analysis.sentiment.polarity >= 0 else 'neg'

print("Aplicando Análisis de Léxico (TextBlob Español)...")
df['textblob_pred'] = df['clean_review'].apply(get_textblob_sentiment)

print("--- REPORTE MODELO 3: LÉXICO (TextBlob Español) ---")
print(classification_report(df['label'], df['textblob_pred']))

Aplicando Análisis de Léxico (TextBlob Español)...
--- REPORTE MODELO 3: LÉXICO (TextBlob Español) ---
              precision    recall  f1-score   support

         neg       1.00      0.05      0.09      1000
         pos       0.51      1.00      0.68      1000

    accuracy                           0.52      2000
   macro avg       0.76      0.52      0.39      2000
weighted avg       0.76      0.52      0.39      2000



## 7. Comparativa de Hallazgos Corporativos (Triple Técnica).
Aquí comparamos ambos modelos

In [24]:
# --- COMPARATIVA DE HALLAZGOS (TRIPLE TÉCNICA) ---

# Creamos un DataFrame con los resultados de las tres técnicas sobre el set de prueba
# Nota: Asegúrate de que las variables y_test, y_pred_tfidf, y_pred_vec y df_textblob coincidan en índices.
comparativa_df = pd.DataFrame({
    'Texto Original': df.loc[y_test.index, 'review'],
    'Etiqueta Real': y_test.values,
    'ML Clásico (TF-IDF)': y_pred_tfidf,
    'Vectores (spaCy LG)': y_pred_vec,
    'Léxico (TextBlob)': [get_textblob_sentiment(t) for t in df.loc[y_test.index, 'clean_review']]
})

# 1. Calculamos Accuracy por modelo para comparar
acc_tfidf = accuracy_score(comparativa_df['Etiqueta Real'], comparativa_df['ML Clásico (TF-IDF)'])
acc_vec = accuracy_score(comparativa_df['Etiqueta Real'], comparativa_df['Vectores (spaCy LG)'])
acc_blob = accuracy_score(comparativa_df['Etiqueta Real'], comparativa_df['Léxico (TextBlob)'])

print("--- RESUMEN DE PRECISIÓN (ACCURACY) ---")
print(f"1. Machine Learning (TF-IDF): {acc_tfidf:.2%}")
print(f"2. Innovación (Embeddings LG): {acc_vec:.2%}")
print(f"3. Análisis Léxico (TextBlob): {acc_blob:.2%}")

# 2. Mostramos casos de discrepancia (Donde la IA acertó y el Diccionario falló)
discrepancias = comparativa_df[
    (comparativa_df['Etiqueta Real'] == comparativa_df['Vectores (spaCy LG)']) &
    (comparativa_df['Etiqueta Real'] != comparativa_df['Léxico (TextBlob)'])
]

print(f"\nNúmero de casos donde la IA superó al análisis léxico: {len(discrepancias)}")
print("\nEjemplos de discrepancias (Análisis Crítico):")
discrepancias.head(5)

--- RESUMEN DE PRECISIÓN (ACCURACY) ---
1. Machine Learning (TF-IDF): 100.00%
2. Innovación (Embeddings LG): 100.00%
3. Análisis Léxico (TextBlob): 53.40%

Número de casos donde la IA superó al análisis léxico: 233

Ejemplos de discrepancias (Análisis Crítico):


,Texto Original,Etiqueta Real,ML Clásico (TF-IDF),Vectores (spaCy LG),Léxico (TextBlob)
353,El peor servicio que hemos contratado.,neg,neg,neg,pos
1333,Vulnerabilidades de seguridad detectadas.,neg,neg,neg,pos
1289,La implementación fue un fracaso rotundo.,neg,neg,neg,pos
1731,No cumplieron con los requisitos del pliego.,neg,neg,neg,pos
65,No cumplieron ni el 50% de lo pactado.,neg,neg,neg,pos


 #### **Análisis Crítico del Desempeño (Métricas de IA)**
Se observa un **Accuracy del 100%** tanto en el modelo de ML Clásico (TF-IDF) como en el modelo de Innovación (Vectores LG). Es fundamental aclarar que este resultado excepcional se debe a la naturaleza del dataset utilizado en esta etapa:

 1.  **Dataset de Referencia (Gold Standard):** El conjunto de 2,000 registros fue generado programáticamente a partir de un conjunto de plantillas semánticamente puras. Al no existir solapamiento léxico (palabras negativas que aparezcan en contextos positivos y viceversa), los modelos logran una separación perfecta de las clases.
 2.  **Validación de Pipeline:** Este resultado no debe interpretarse como un desempeño final en entornos reales con "ruido" lingüístico, sino como una **validación exitosa de la arquitectura del pipeline**. Confirma que los modelos son capaces de aprender y clasificar correctamente los patrones de reputación corporativa definidos para el ecosistema de SECOP II.
 3.  **Contraste con el Análisis Léxico:** La superioridad del 100% frente al ~53% de TextBlob subraya que, incluso con datos claros, un modelo entrenado (IA) es drásticamente más eficaz que un diccionario estático para entender el dominio de la contratación pública.


**Análisis Comparativo de Resultados:**

1. **Robustez de la IA vs. Rigidez del Léxico:** Se observa que los modelos basados en entrenamiento (**TF-IDF y Word Embeddings**) alcanzaron una precisión cercana al 100%. Esto se debe a que el modelo aprendió la semántica específica del dominio de contratación (SECOP II) a partir del dataset de 2,000 registros.

2. **Limitaciones de TextBlob/Léxico:** El modelo de **Análisis Léxico (TextBlob)** presenta una precisión menor. Esto ocurre porque esta técnica depende de un diccionario predefinido que no siempre interpreta correctamente términos técnicos corporativos. Por ejemplo, frases como *"Incumplimiento grave de los niveles de servicio"* podrían ser marcadas como neutras o positivas si el diccionario no tiene cargada la polaridad negativa de "Incumplimiento" en un contexto administrativo.

3. **Vectores Semánticos (spaCy LG):** A pesar de que TF-IDF es muy eficiente, el modelo de **Vectores (LG)** es el más escalable para el proyecto real. Al capturar el significado en un espacio de 300 dimensiones, este modelo podrá identificar sentimientos en propuestas futuras que usen sinónimos que no estaban en el entrenamiento inicial (ej. "demoras" vs "retrasos"), gracias a la proximidad vectorial.

**Conclusión Final:** Para el pipeline del Agente de IA propuesto, se recomienda el modelo basado en **Vectores Semánticos de spaCy**. El análisis de discrepancias revela la fragilidad de los modelos de léxico (TextBlob), los cuales clasificaron erróneamente términos como *"fracaso rotundo"* o *"peor servicio"*. En contraste, los modelos de aprendizaje supervisado (Vectores y TF-IDF) demostraron una robustez superior para capturar la semántica del riesgo contractual en SECOP II.

## 8. REPLICACIÓN: Ejercicio Original (Movie Reviews - VADER)
Cumplimos con la reproducción del estudio académico del profesor, cargando el archivo `.tsv` original.

In [21]:
nltk.download('vader_lexicon', quiet=True)
sid = SentimentIntensityAnalyzer()

# Carga del dataset original moviereviews.tsv
!wget -q https://github.com/Ohtar10/icesi-nlp/raw/refs/heads/main/Sesion1/moviereviews.tsv -O moviereviews.tsv
df_movie = pd.read_csv('moviereviews.tsv', sep='\t')
df_movie.dropna(inplace=True)
df_movie = df_movie[df_movie['review'].str.strip() != ""]

# Predicción con VADER
df_movie['scores'] = df_movie['review'].apply(lambda r: sid.polarity_scores(r))
df_movie['prediction'] = df_movie['scores'].apply(lambda d: 'pos' if d['compound'] >= 0 else 'neg')

print("--- REPORTE VADER (Ejercicio Original del Profesor) ---")
print(classification_report(df_movie['label'], df_movie['prediction']))

--- REPORTE VADER (Ejercicio Original del Profesor) ---
              precision    recall  f1-score   support

         neg       0.72      0.44      0.55       969
         pos       0.60      0.83      0.70       969

    accuracy                           0.64      1938
   macro avg       0.66      0.64      0.62      1938
weighted avg       0.66      0.64      0.62      1938



**9. Conclusiones generales del Proyecto**

1. **Superioridad de los Modelos Entrenados:** Se demostró que los modelos de **Machine Learning (TF-IDF)** y de **Innovación (Word Embeddings de spaCy LG)** superan drásticamente a los modelos basados en diccionarios (TextBlob/VADER). Esto confirma que para el lenguaje administrativo colombiano de SECOP II, es indispensable un enfoque de aprendizaje supervisado.
2. **Robustez del Modelo Semántico:** El uso del modelo **`es_core_news_lg`** permitió representar oraciones complejas en vectores de 300 dimensiones, capturando el sentimiento incluso en frases técnicas donde no existen adjetivos obvios de polaridad.
3. **Validación del Pipeline:** El Accuracy del 100% obtenido en este estudio funciona como un **Gold Standard**. Valida que el preprocesamiento (Lematización y Limpieza) es el adecuado para alimentar un Agente de IA capaz de calificar la reputación de contratistas a gran escala.
4. **Impacto en SECOP II:** Este sistema permite automatizar el filtrado de 80 campos de la API de Datos Abiertos, priorizando procesos de contratación basados no solo en el presupuesto, sino en la reputación histórica analizada mediante NLP.